In [1]:
import torch
from sd_vae.ae import VAE
# from sd_vae.vae import Encoder
%load_ext autoreload
%autoreload 2

In [6]:
vae = VAE(
    channels=32,
    channel_multipliers=[1, 2, 4],
    n_resnet_blocks=1,
    x_channels=1,
    z_channels=4,
    norm_channels=32,
    n_heads=4,
).cuda()

In [8]:
x = torch.rand(2, 1, 32, 32).cuda()


In [10]:
xhat, posterior = vae(x)

In [12]:
posterior.sample().shape

torch.Size([2, 4, 8, 8])